In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

import numpy as np
import torch
import librosa
from torch.nn import functional as F
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

from wavenet import model, train, sample, audio, datasets, utils, viz

# Train on Sines

A very simple dataset, should be able to crush this.

In [ ]:
dry = False
if dry:
    os.environ['WANDB_MODE'] = 'dryrun'

In [ ]:
p = model.HParams(n_audio_chans=2)
X, X_test = datasets.Sines(60 * 60 * 10, 1, p), datasets.Sines(60 * 60 * 2, 1, p)
batch = datasets.to_tensor(X, 20)

In [ ]:
m = model.Wavenet(p)
t = train.Trainer(m, X, X_test, train.HParams(max_epochs=100, batch_size=16, num_workers=10, learning_rate=0.0008), None)

In [ ]:
track_i = viz.plot_track(batch, n_samples=p.receptive_field_size())

In [ ]:
ipd.Audio(audio.mu_expand(batch[track_i].numpy(), p), rate=p.sampling_rate)

In [ ]:
t.train()

In [ ]:
_, track = sample.sample(m, decoder=utils.decode_nucleus(), n_samples=32000, batch_size=10)

In [ ]:
viz.plot_track(track, n_samples=p.receptive_field_size() // 6)

In [ ]:
ipd.Audio(track[0], rate=p.sampling_rate)